In [0]:
!pip install -U -q PyDrive
!ls

sample_data


In [0]:
from google.colab import drive
drive.mount('/drive/')

In [0]:
import zipfile
#this does not work for nested folders .i.e. if there is folder in folder in 
#folder in the zip file
zip_ref = zipfile.ZipFile("xx/Colab_folder/train.zip", 'r')
zip_ref.extractall("xx/Colab_folder/")
zip_ref.close()

zip_ref = zipfile.ZipFile("xx/Colab_folder/test1.zip", 'r')
zip_ref.extractall("xx/Colab_folder/")
zip_ref.close()

In [0]:
!pip install tqdm

In [0]:
import os
import shutil
from random import shuffle
from tqdm import tqdm_notebook
def organize_datasets(path_to_data, n=4000, ratio=0.2):
    files = os.listdir(path_to_data)
    files = [os.path.join(path_to_data, f) for f in files]
    shuffle(files)
    files = files[:n]
    
    n = int(len(files) * ratio)
    val, train = files[:n], files[n:]


#    shutil.rmtree('./train/')
#    print('/train/ removed')

    for c in ['dogs', 'cats']: 
        os.makedirs('xx/Colab_folder/train/{0}/'.format(c))
        os.makedirs('xx/Colab_folder/validation/{0}/'.format(c))

    print('folders created !')

    for t in tqdm_notebook(train):
        if 'cat' in t:
            shutil.copy2(t, os.path.join('xx', 'Colab_folder','train', 'cats'))
        else:
            shutil.copy2(t, os.path.join('xx', 'Colab_folder', 'train', 'dogs'))
     
    for v in tqdm_notebook(val):
        if 'cat' in v:
            shutil.copy2(v, os.path.join('xx', 'Colab_folder', 'validation', 'cats'))
        else:
            shutil.copy2(v, os.path.join('xx', 'Colab_folder', 'validation', 'dogs'))
            
    print('Data copied!')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import Callback

ratio = 0.2
n = 25000

batch_size = 32

#organize_datasets('./xx/Colab_folder/train/',n,ratio)

print('training data Image generator')
train_datagen = ImageDataGenerator(rescale=1/255.,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True
                                    )
print('validating data image generator')
val_datagen = ImageDataGenerator(rescale=1/255.)

print('getting train_generator')
train_generator = train_datagen.flow_from_directory(
        'xx/Colab_folder/train/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

print('getting validation_generator')
validation_generator = val_datagen.flow_from_directory(
        'xx/Colab_folder/validation/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

print('setting up model')
model = Sequential()

print('setting 1st convolutional layer')
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), padding='same', activation='relu'))
print('maxpooling')
model.add(MaxPooling2D(pool_size=(2, 2)))

print('setting 2nd convolutional layer')
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
print('maxpooling')
model.add(MaxPooling2D(pool_size=(2, 2)))

print('setting 3rd convolutional layer')
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
print('maxpooling')
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

epochs = 50
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

history = LossHistory()

## Callback for early stopping the training
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

fitted_model = model.fit_generator(
        train_generator,
        steps_per_epoch= int(n * (1-ratio)) // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps= int(n * ratio) // batch_size,
        )

training data Image generator
validating data image generator
getting train_generator
Found 20000 images belonging to 2 classes.
getting validation_generator
Found 5000 images belonging to 2 classes.
setting up model
setting 1st convolutional layer
maxpooling
setting 2nd convolutional layer
maxpooling
setting 2nd convolutional layer
maxpooling
Epoch 1/10
625/625 [==============================] - 315s 503ms/step - loss: 0.6876 - acc: 0.5422 - val_loss: 0.6841 - val_acc: 0.5392
Epoch 2/10
625/625 [==============================] - 301s 481ms/step - loss: 0.6484 - acc: 0.6220 - val_loss: 0.5876 - val_acc: 0.6934
Epoch 3/10
625/625 [==============================] - 301s 482ms/step - loss: 0.5838 - acc: 0.6872 - val_loss: 0.5384 - val_acc: 0.7318
Epoch 4/10
625/625 [==============================] - 305s 488ms/step - loss: 0.5455 - acc: 0.7255 - val_loss: 0.5008 - val_acc: 0.7622
Epoch 5/10
625/625 [==============================] - 309s 495ms/step - loss: 0.5158 - acc: 0.7461 - val_loss:

In [0]:
!fusermount -u drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
!ls "xx"/Colab_folder

test1.zip  train.zip


In [0]:
!mkdir ~/Colab_folder/train

mkdir: cannot create directory ‘/root/Colab_folder/train’: No such file or directory


In [0]:
!ls 'xx/Colab_folder/train'

In [0]:
shutil.rmtree("xx/Colab_folder/train/dogs")